In [ ]:
import torch
from torch import nn
import torchsummary

In [ ]:
class SE_Block(nn.Module):                         # Squeeze-and-Excitation block
    def __init__(self, in_channels):
        super(SE_Block, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))     # GAP
        self.conv1 = nn.Conv2d(in_channels, in_channels // 16, kernel_size=1)   # 1x1的卷积核充当FC
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels // 16, in_channels, kernel_size=1)   # 1x1的卷积核充当FC
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        out = self.sigmoid(x)
        return out


In [ ]:
class BasicBlock(nn.Module):      # 左侧的 residual block 结构（18-layer、34-layer）

    # 如果是BasicBlock，每个小block的输入=输出
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1):      # 两层卷积 Conv2d + Shutcuts
        super(BasicBlock, self).__init__()
        
        # 两个3x3卷积核
        self.conv = nn.Sequential(
            # 第一个cov是用来改变WxH的（stride可指定）
            ConvBN(in_channels=in_channels, out_channels=out_channels, 
                       kernel_size=3, stride=stride, padding=1),
            
            nn.ReLU(inplace=True),
            
            # 第二个conv的stride恒为1，不改变WxH
            ConvBN(in_channels=out_channels, out_channels=out_channels, 
                   kernel_size=3, stride=1, padding=1),     
        )
        
        # 新增的SE-Block
        self.SE = SE_Block(out_channels)           # Squeeze-and-Excitation block

        self.shortcut = nn.Sequential()
        
        # Shortcuts用于构建 Conv Block 和 Identity Block
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                # 卷积+BN，不激活
                ConvBN(in_channels=in_channels, out_channels=self.expansion*out_channels, 
                   kernel_size=1, stride=stride)
            )
            
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv(x)
        
        SE_out = self.SE(out)   # 经过Residual-block后再经过SE-block
        
        out = out * SE_out      # 对位相乘，重新加权
        
        out += self.shortcut(x)
        return self.relu(out)


In [ ]:
class Bottleneck(nn.Module):      # 右侧的 residual block 结构（50-layer、101-layer、152-layer）
    
    # 观察50-layer可以发现，各个block内部卷积核通道数是4倍的关系
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride=1):      # 三层卷积 Conv2d + Shutcuts
        super(Bottleneck, self).__init__()
        
        # 1x1 -> 3x3 -> 1x1
        self.conv = nn.Sequential(
            # 第一个cov是用来降维的，减少参数量
            ConvBN(in_channels=in_channels, out_channels=out_channels, 
                   kernel_size=1),
            nn.ReLU(inplace=True),
            
            # 第二个conv是用来改变WxH的（stride可指定）
            ConvBN(in_channels=out_channels, out_channels=out_channels, 
                   kernel_size=3, stride=stride, padding=1),
            nn.ReLU(inplace=True),  
            
            # 第三个conv用来升维
            ConvBN(in_channels=out_channels, out_channels=self.expansion*out_channels, 
                   kernel_size=1)       
        )  
        
        self.SE = SE_Block(self.expansion*out_channels)           # Squeeze-and-Excitation block
        
        self.shortcut = nn.Sequential()
        
        # Shortcuts用于构建 Conv Block 和 Identity Block
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                # 卷积+BN，不激活
                ConvBN(in_channels=in_channels, out_channels=self.expansion*out_channels, 
                   kernel_size=1, stride=stride)
            )
            
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.conv(x)
        
        SE_out = self.SE(out)   # 经过Residual-block后再经过SE-block
        
        out = out * SE_out      # 对位相乘，重新加权
        
        out += self.shortcut(x)
        return self.relu(out)


In [ ]:
class SE_ResNet(nn.Module):
    def __init__(self, block, numlist_blocks, num_classes=2):
        """
        Args:
            block:      选用BasicBlock还是Bottleneck这两种残差结构
            num_blocks: 针对不同数量的layers，有不同的组合，比如ResNet50为[3, 4, 6, 3]
            num_classes:最终分类数量
        """
        super(SE_ResNet, self).__init__()
        
        self.in_channels = 64

        # 原始输入为229x229x3 -> 112x112x64
        self.conv1 = ConvBN(in_channels=3, out_channels=64, kernel_size=7, stride=2) # conv1
        # 112x112x64 -> 56x56x64
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)         # maxpool
        
        self.layer1 = self._make_layer(block, 64,  numlist_blocks[0], stride=1)      # conv2_x
        self.layer2 = self._make_layer(block, 128, numlist_blocks[1], stride=2)      # conv3_x
        self.layer3 = self._make_layer(block, 256, numlist_blocks[2], stride=2)      # conv4_x
        self.layer4 = self._make_layer(block, 512, numlist_blocks[3], stride=2)      # conv5_x
        
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))     # 平均池化
        self.linear = nn.Linear(2048, num_classes)      # 线性层
        self.relu = nn.ReLU(inplace=True)

    def _make_layer(self, block, in_channels, num_blocks, stride):
        # 虽然每个convn_x由多个block组成，但是其中只有某个block的stride为2，剩余的为1
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        
        for stride in strides:
            layers.append(block(self.in_channels, in_channels, stride))
            
            # 经过某个convn_x之后，in_channels被放大对应expansion倍
            self.in_channels = in_channels * block.expansion
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.conv1(x))    # conv1
        x = self.maxpool(x)             # maxpool
        x = self.layer1(x)              # conv2_x
        x = self.layer2(x)              # conv3_x
        x = self.layer3(x)              # conv4_x
        x = self.layer4(x)              # conv5_x
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        out = self.linear(x)
        
        return out


In [ ]:
def make_net(net, num_classes):
    if net == 'SE_ResNet18':
        return SE_ResNet(BasicBlock, [2, 2, 2, 2], num_classes)
    if net == 'SE_ResNet34':
        return SE_ResNet(BasicBlock, [3, 4, 6, 3], num_classes)
    if net == 'SE_ResNet50':
        return SE_ResNet(Bottleneck, [3, 4, 6, 3], num_classes)
    if net == 'SE_ResNet101':
        return SE_ResNet(Bottleneck, [3, 4, 23, 3], num_classes)
    if net == 'SE_ResNet152':
        return SE_ResNet(Bottleneck, [3, 8, 36, 3], num_classes)
        
def test():
    SE_ResNet50 = make_net('SE_ResNet50', num_classes=2)
    #创建模型，部署gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    SE_ResNet50.to(device)
    summary(SE_ResNet50, (3, 229, 229))
    
test()
